# Step 1: Creating Tokens

In [12]:
with open("the-verdict.txt", "r") as book:
    raw_text = book.read()

print("The number of characters in the book are:", len(raw_text))
print(raw_text[:99])

The number of characters in the book are: 20395
I had always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [14]:
import re

text = "Hello world! This is a test."
result = re.split(r'(\s)', text)

print(result)

['Hello', ' ', 'world!', ' ', 'This', ' ', 'is', ' ', 'a', ' ', 'test.']


In [15]:
text = "Hello world! This is sample--text for testing."
result = re.split(r"([,.:;?_!\"()\']|--|\s)", text)

# Remove the whitespaces from the list
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', 'world', '!', 'This', 'is', 'sample', '--', 'text', 'for', 'testing', '.']


## Converting the raw text of the book into tokens

In [16]:
preprocessed = result = re.split(r"([,.:;?_!\"()\']|--|\s)", raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

['I', 'had', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [17]:
print("The lenght of tokens is:", len(preprocessed))

The lenght of tokens is: 4663


# Step 2: Creating Token ID's

In [19]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)
print("The size of the vocabulary is:", vocab_size)

The size of the vocabulary is: 1145


In [22]:
print(all_words[:15])

['!', '"', "'", '(', ')', ',', '--', '.', ':', ';', '?', 'A', 'AM', 'Ah', 'Among']


In [25]:
vocabulary = {token:integer for integer, token in enumerate(all_words)}

In [27]:
for i, item in enumerate(vocabulary.items()):
    print(item)
    if i>=50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('AM', 12)
('Ah', 13)
('Among', 14)
('And', 15)
('Are', 16)
('Arrt', 17)
('As', 18)
('At', 19)
('Be', 20)
('Begin', 21)
('Burlington', 22)
('But', 23)
('By', 24)
('Carlo', 25)
('Chicago', 26)
('Claude', 27)
('Come', 28)
('Croft', 29)
('Destroyed', 30)
('Devonshire', 31)
('Don', 32)
('Dubarry', 33)
('Emperors', 34)
('End', 35)
('FELT', 36)
('Florence', 37)
('For', 38)
('Gallery', 39)
('Gideon', 40)
('Gisburn', 41)
('Gisburns', 42)
('Grafton', 43)
('Greek', 44)
('Grindle', 45)
('Grindles', 46)
('HAD', 47)
('HAS', 48)
('HAVE', 49)
('Had', 50)


# Implementing the SimpleTokenizerV1 class

In [42]:
class SimpleTokenizerV1:
    def __init__(self, vocabulary):
        self.str_to_int = vocabulary
        self.int_to_str = {index:string for string, index in vocabulary.items()}

    def encode(self, text):
        preprocessed = re.split(r"([,.:;?_!\"()\']|--|\s)", text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]

        ids = [self.str_to_int[string] for string in preprocessed]

        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [43]:
tokenizer = SimpleTokenizerV1(vocabulary)

text = """"The height of his glory"--that was what the women called it.
            I can hear Mrs. Gideon Thwing--his last Chicago sitter--deploring"""

# Encoding the text
ids = tokenizer.encode(text)
print(ids)

[1, 105, 552, 737, 563, 510, 1, 6, 1002, 1092, 1104, 1003, 1127, 255, 600, 7, 58, 257, 549, 74, 7, 40, 112, 6, 563, 616, 26, 912, 6, 340]


In [44]:
# Decoding the text
print(tokenizer.decode(ids))

" The height of his glory" -- that was what the women called it. I can hear Mrs. Gideon Thwing -- his last Chicago sitter -- deploring


# Implementing The Special Context Tokens to handle the tokens which are not present in the vocabulary

In [59]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|unk|>", "<|endoftext|>"])

vocabulary = {token:integer for integer, token in enumerate(all_tokens)}

len(vocabulary)

1147

In [60]:
for i, item in enumerate(list(vocabulary.items())[-5:]):
    print(item)

('younger', 1142)
('your', 1143)
('yourself', 1144)
('<|unk|>', 1145)
('<|endoftext|>', 1146)


In [87]:
class SimpleTokenizerV2:
    def __init__(self, vocabulary):
        self.str_to_int = vocabulary
        self.int_to_str = {index:string for string, index in vocabulary.items()}

    def encode(self, text):
        preprocessed = re.split(r"([,.:;?_!\"()\']|--|\s)", text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]

        preprocessed = [
            token if token in self.str_to_int 
            else "<|unk|>" for token in preprocessed
        ]

        ids = [self.str_to_int[token] for token in preprocessed]

        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [88]:
tokenizer = SimpleTokenizerV2(vocabulary)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join([text1, text2])

print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [89]:
# Encoding this text
ids = tokenizer.encode(text)
print(ids)

[1145, 5, 369, 1141, 642, 990, 10, 1146, 60, 1003, 971, 999, 737, 1003, 1145, 7]


In [90]:
# Decoding the ids
print(tokenizer.decode(ids))

<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.
